In [192]:
import pandas as pd
import scipy.constants
from scipy.optimize import curve_fit
from scipy.integrate import quad
from scipy.signal import argrelextrema
from sklearn.metrics import r2_score
#import seaborn
#seaborn.set(style='whitegrid')
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.dates as md
import matplotlib.transforms as transforms
from matplotlib.ticker import LogFormatter
%matplotlib qt
# %matplotlib inline
import datetime as dt
pd.set_option('display.max_rows', 100) 


## Reading in the data

In [193]:
#read in the data
mid = pd.read_csv("MID.tsv", sep="\t", skiprows=lambda x: x <=11,  on_bad_lines="skip")
mid.drop([0,1], inplace=True)
print(mid.describe())
#bring the time data into a new df
time = pd.DataFrame(mid.iloc[:,0::2]) #- get every second column
print(time.describe())
#add elapsed time columns
def elapsed_time(df):
    itr = 0
    counter = len(df.columns)
    masses = [2,4,5.5,12,15,16,18,28,32,40,44]
    for i in df:
        timestamp = pd.to_datetime(df[i], format="%Y/%m/%d %H:%M:%S.%f")
        runtime = (timestamp-timestamp[2]).dt.total_seconds()
        colname = "Elapsed time, mass {}".format(masses[itr]) 
        df.insert(counter, colname, runtime)
        itr += 1
        counter += 1
    return(df)
elapsed_time(time)

time_sliced = time.iloc[:,11:].fillna(0)
print(time_sliced)

#convert elapsed times in time-dataframe back to a timestamp
def h2_time(df1,df2):
    startdate = pd.to_datetime(df1.iloc[2,0],format="%Y/%m/%d %H:%M:%S.%f")
    dic= {}
    itr = 0
    masses = [2,4,5.5,12,15,16,18,28,32,40,44]
    key = "Timestamp, mass {}".format(masses[itr])
    for label,content in df2.items():
        key = "Timestamp, mass {}".format(masses[itr])
        #.strftime('%d-%m-%Y %H:%M:%S') not used
        for row in content:
            dic.setdefault(key,[]).append((startdate+dt.timedelta(milliseconds=row*1000)))
        itr += 1
    new_df = pd.DataFrame.from_dict(dic)
    return(new_df)

new_df = h2_time(time,time_sliced)
new_df


       mass 2.0_value  mass 4.0_value  mass 5.5_value  mass 12.0_value  \
count    1.697000e+03    1.697000e+03    1.697000e+03     7.600000e+02   
mean     2.037199e-10    4.601856e-13    5.874125e-14     7.766868e-14   
std      5.271121e-10    2.251720e-13    6.557085e-14     2.285582e-13   
min      2.820090e-11    0.000000e+00    1.800000e-15     0.000000e+00   
25%      4.140030e-11    2.060000e-13    2.040000e-14     6.675000e-15   
50%      5.151870e-11    5.804000e-13    4.270000e-14     1.470000e-14   
75%      2.524410e-10    6.435000e-13    6.420000e-14     3.325000e-14   
max      5.392830e-09    7.842000e-13    5.939000e-13     1.490600e-12   

       mass 15.0_value  mass 16.0_value  mass 18.0_value  mass 28.0_value  \
count     5.000000e+01     7.000000e+00     5.000000e+00     5.000000e+00   
mean      4.224200e-13     6.674286e-14     3.599200e-13     2.245600e-13   
std       2.209199e-13     7.664065e-14     5.783759e-14     6.517793e-14   
min       1.920000e-14   

,"Timestamp, mass 2","Timestamp, mass 4","Timestamp, mass 5.5","Timestamp, mass 12","Timestamp, mass 15","Timestamp, mass 16","Timestamp, mass 18","Timestamp, mass 28","Timestamp, mass 32","Timestamp, mass 40","Timestamp, mass 44"
0,2023-01-13 14:48:28.655,2023-01-13 14:48:28.655,2023-01-13 14:48:28.655,2023-01-13 14:48:28.655,2023-01-13 14:48:28.655,2023-01-13 14:48:28.655,2023-01-13 14:48:28.655,2023-01-13 14:48:28.655,2023-01-13 14:48:28.655,2023-01-13 14:48:28.655,2023-01-13 14:48:28.655
1,2023-01-13 14:48:35.051,2023-01-13 14:48:35.051,2023-01-13 14:48:35.051,2023-01-13 14:48:35.051,2023-01-13 14:48:35.050,2023-01-13 14:48:35.050,2023-01-13 14:48:35.050,2023-01-13 14:48:35.051,2023-01-13 14:48:35.051,2023-01-13 14:48:35.051,2023-01-13 14:48:28.655
2,2023-01-13 14:48:41.446,2023-01-13 14:48:41.446,2023-01-13 14:48:41.446,2023-01-13 14:48:41.446,2023-01-13 14:48:41.445,2023-01-13 14:48:41.445,2023-01-13 14:48:41.445,2023-01-13 14:48:41.446,2023-01-13 14:48:41.446,2023-01-13 14:48:41.446,2023-01-13 14:48:28.655
3,2023-01-13 14:48:47.841,2023-01-13 14:48:47.841,2023-01-13 14:48:47.841,2023-01-13 14:48:47.841,2023-01-13 14:48:47.840,2023-01-13 14:48:47.840,2023-01-13 14:48:47.840,2023-01-13 14:48:47.841,2023-01-13 14:48:47.841,2023-01-13 14:48:47.841,2023-01-13 14:48:28.655
4,2023-01-13 14:48:54.236,2023-01-13 14:48:54.236,2023-01-13 14:48:54.236,2023-01-13 14:48:54.236,2023-01-13 14:48:54.235,2023-01-13 14:48:54.235,2023-01-13 14:48:54.235,2023-01-13 14:48:54.236,2023-01-13 14:48:54.236,2023-01-13 14:48:28.655,2023-01-13 14:48:28.655
...,...,...,...,...,...,...,...,...,...,...,...
1692,2023-01-13 17:48:52.037,2023-01-13 17:48:49.036,2023-01-13 17:48:50.037,2023-01-13 14:48:28.655,2023-01-13 14:48:28.655,2023-01-13 14:48:28.655,2023-01-13 14:48:28.655,2023-01-13 14:48:28.655,2023-01-13 14:48:28.655,2023-01-13 14:48:28.655,2023-01-13 14:48:28.655
1693,2023-01-13 17:48:58.432,2023-01-13 17:48:55.431,2023-01-13 17:48:56.432,2023-01-13 14:48:28.655,2023-01-13 14:48:28.655,2023-01-13 14:48:28.655,2023-01-13 14:48:28.655,2023-01-13 14:48:28.655,2023-01-13 14:48:28.655,2023-01-13 14:48:28.655,2023-01-13 14:48:28.655
1694,2023-01-13 17:49:04.827,2023-01-13 17:49:01.826,2023-01-13 17:49:02.827,2023-01-13 14:48:28.655,2023-01-13 14:48:28.655,2023-01-13 14:48:28.655,2023-01-13 14:48:28.655,2023-01-13 14:48:28.655,2023-01-13 14:48:28.655,2023-01-13 14:48:28.655,2023-01-13 14:48:28.655
1695,2023-01-13 17:49:11.222,2023-01-13 17:49:08.221,2023-01-13 17:49:09.222,2023-01-13 14:48:28.655,2023-01-13 14:48:28.655,2023-01-13 14:48:28.655,2023-01-13 14:48:28.655,2023-01-13 14:48:28.655,2023-01-13 14:48:28.655,2023-01-13 14:48:28.655,2023-01-13 14:48:28.655


In [194]:
#reading the Emission current, pressures
vaclog=pd.read_csv("vaclog", sep="\t")
vaclog.head()
#adding an elapsed time column
timestamp=[]
for i in vaclog["Time"]:
    timestamp.append(i)
#making the column into a datetime object
timestamps=pd.to_datetime(vaclog["Time"],format="%d/%m/%Y %H:%M:%S")
runtime=(timestamps-timestamps[0]).dt.total_seconds()    
vaclog.insert(2,"Elapsed time",runtime)
vaclog.drop(index=vaclog.index[0], 
        axis=0, 
        inplace=True)

vaclog.head()

,Live comments,Time,Elapsed time,injection 100mbar,Barion_2,Barion_1,DUAL experiment,DUAL insulation,injection 1mbar,helium,T-platinum,T-CERNOX,I_emission,I_grid
1,NaN,13/01/2023 09:37:08,8.0,0.006098,0.0,0.0,0.000002,4.539000e-07,0.22708,0.0,-25.589,195.397,NaN,NaN
2,NaN,13/01/2023 09:37:15,15.0,0.006244,0.0,0.0,0.000002,4.544000e-07,0.22708,0.0,-25.589,195.285,NaN,NaN
3,NaN,13/01/2023 09:37:22,22.0,0.005826,0.0,0.0,0.000002,4.539000e-07,0.22708,0.0,-25.589,195.397,NaN,NaN
4,NaN,13/01/2023 09:37:29,29.0,0.005948,0.0,0.0,0.000002,4.544000e-07,0.22709,0.0,-25.926,195.397,NaN,NaN
5,NaN,13/01/2023 09:37:36,36.0,0.006393,0.0,0.0,0.000002,4.544000e-07,0.22708,0.0,-25.814,195.285,NaN,NaN


In [195]:
#Reading in the HiVolta measurement file 
hv = pd.read_csv("hv log", sep=",")

#hv.drop(columns=["Vmon1","Vmon2","Vmon3","Vmon4","Vmon5","Vmon6","Vmon7","Vmon8","Imon3","Imon4","Imon5","Imon6","Imon7","Imon8","Comments1"], inplace=True)
hv["hv_grid"]=[element * 1e-6 for element in hv["IMon1"]]
hv["I_em"]=[element * 1e-6 for element in hv["IMon2"]]

# hv.drop(hv.columns[hv.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
hv["I_em"]=abs(hv["I_em"])
hv["hv_grid"]=abs(hv["hv_grid"])

#adding a datetime column
datetime = pd.to_datetime(hv['Date'] + ' ' + hv['Time'],format="%d/%m/%Y %H:%M:%S.%f")
datetime = datetime.dt.strftime('%d-%m-%Y %H:%M:%S')
datetime = pd.to_datetime(datetime, format='%d-%m-%Y %H:%M:%S')
hv.insert(0,"datetime",datetime)
    
#making the column into a datetime object
hv_timestamps=pd.to_datetime(hv["Time"],format="%H:%M:%S.%f")
runtime=(hv_timestamps-hv_timestamps[0]).dt.total_seconds()    
hv.insert(2,"Elapsed time",runtime)

hv.head()



,datetime,Date,Elapsed time,Time,VMon1,VMon2,VMon3,VMon4,VMon5,VMon6,...,IMon3,IMon4,IMon5,IMon6,IMon7,IMon8,Comment,Unnamed: 19,hv_grid,I_em
0,2023-01-13 14:50:55,13/01/2023,0.000,14:50:55.409,0.02,0.06,0.04,0.08,0.0,0.0,...,0.0087,0.008,0.0029,0.009,-0.0216,-0.0125,NaN,NaN,9.000000e-10,8.100000e-09
1,2023-01-13 14:50:56,13/01/2023,1.185,14:50:56.594,0.02,0.06,0.04,0.08,0.0,0.0,...,0.0087,0.008,0.0026,0.009,-0.0216,-0.0125,NaN,NaN,9.000000e-10,8.100000e-09
2,2023-01-13 14:50:57,13/01/2023,2.358,14:50:57.767,0.02,0.06,0.04,0.08,0.0,0.0,...,0.0087,0.008,0.0023,0.009,-0.0216,-0.0125,NaN,NaN,9.000000e-10,8.100000e-09
3,2023-01-13 14:50:58,13/01/2023,3.544,14:50:58.953,0.02,0.06,0.04,0.08,0.0,0.0,...,0.0087,0.008,0.0021,0.009,-0.0216,-0.0125,NaN,NaN,9.000000e-10,8.100000e-09
4,2023-01-13 14:51:00,13/01/2023,4.726,14:51:00.135,0.02,0.06,0.04,0.08,0.0,0.0,...,0.0087,0.008,0.0020,0.009,-0.0216,-0.0125,NaN,NaN,9.000000e-10,8.100000e-09


### Data processing

In [196]:
#print vaclog comments
print(pd.unique(vaclog["Live comments"]))

[nan 'inj volume prepared' 'starting injection']


In [197]:
#hv comments
print(pd.unique(hv["Comment"]))
hv_comm = hv.loc[:,["datetime","Comment"]]
hv_comm.dropna(inplace=True)
hv_comm.index

[nan 'no emission' 'electron activity & grid repelling' 'grid transparent'
 'grid repelling']


Int64Index([34, 35, 36, 37, 167, 168, 169, 170, 196, 197, 198, 199, 6538, 6539,
            6540, 6541],
           dtype='int64')

In [198]:
#Temperature curve for CERNOX - for temp stability
A=[230.317302,-6170.1513,71837.9529,-477946.76,2.003668910085786e+6,-5.488690193047771e+6,9.830475663897528e+6,-1.111226817786569e+7,7.202477878914065e+6,-2.04194551328507e+6]
#specify fit parameters A, data (Resistance values)
def polyfit(params,data):
    total=[]
    for j in data: 
        exp=0
        for i in range(len(params)):
            exp += (params[i]/(math.log10(j))**i)
        total.append(10**exp)
    return(total)    


In [199]:
#Filtering background signal

#Find the timestamp where emission was turned on and off
print("Result 1:",hv.loc[hv["Comment"]=="grid transparent"])
print("Result 2:", hv.loc[hv["Comment"]=="grid repelling"])
print("Result 3:", hv.loc[hv["Comment"]=="electron activity grid repelling"])

x=pd.Timestamp(2023, 1, 12, 17 ,23, 58)
y=pd.Timestamp(2023, 1, 12, 17, 28, 43)
z=pd.Timestamp(2023, 1, 12, 17, 20, 13)
#then find the time value closest to it in the MID scan file 
print("\n","Result 4:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - x).abs().argsort(),:])
print("\n","Result 5:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - y).abs().argsort(),:])
print("\n","Result 6:",mid.iloc[(pd.to_datetime(mid['mass 2.0_time'],format="%Y/%m/%d %H:%M:%S") - z).abs().argsort(),:])

#take a mass with a signal known to be in the background
bck=np.mean(mid["mass 5.5_value"])

#apply background correction to the rest of the mass spectrum
mid["mass 2, corr"]=mid["mass 2.0_value"]-bck

 
#From the signal, get the partial pressures 
h2_sens=2.13e-2
p_h2=mid["mass 2, corr"][91:136]/h2_sens
print(p_h2.describe())

#baseline for partial pressure delta
base_h2=mid["mass 2, corr"][56:91]/h2_sens
base_h2.describe()

Result 1:                datetime        Date  Elapsed time          Time  VMon1  \
196 2023-01-13 14:54:55  13/01/2023       240.284  14:54:55.693  82.72   
197 2023-01-13 14:54:56  13/01/2023       241.477  14:54:56.886   0.88   
198 2023-01-13 14:54:58  13/01/2023       242.665  14:54:58.074   0.22   
199 2023-01-13 14:54:59  13/01/2023       243.860  14:54:59.269   0.18   

      VMon2   VMon3   VMon4   VMon5   VMon6  ...   IMon3   IMon4   IMon5  \
196  100.00  999.94  999.98  999.90  499.86  ...  6.9864  7.0585  7.0920   
197  100.02  999.92  999.96  999.90  499.86  ...  7.1135  7.1710  7.1058   
198  100.02  999.92  999.96  999.92  499.86  ...  7.1913  7.1925  7.1905   
199  100.02  999.92  999.96  999.92  499.86  ...  7.1995  7.2069  7.1929   

      IMon6   IMon7   IMon8           Comment  Unnamed: 19       hv_grid  \
196  7.1649 -0.0218 -0.0139  grid transparent          NaN  4.289000e-07   
197  7.1242 -0.0218 -0.0139  grid transparent          NaN  3.288000e-07   
198  7.206

count    3.500000e+01
mean     1.693038e-08
std      1.990342e-10
min      1.660105e-08
25%      1.675828e-08
50%      1.695058e-08
75%      1.709023e-08
max      1.727560e-08
Name: mass 2, corr, dtype: float64

### Injection pressure linearity

In [200]:
#defining a function for the straight line
def func(x,a,b):
    return a*x + b
#curve fit for the data
y = vaclog["injection 100mbar"].values
x = (vaclog["Elapsed time"]/3600).values
params, cov = curve_fit(func,x,y)
#straight line parameters
a, b = params
print(f"Fitted line: y = {a:.2f}*x+{b:.2f}")

Fitted line: y = 0.59*x+81.42


In [201]:
#checking inj pressure linearity, graph
fig, ax = plt.subplots()
y_fit = func(x,a,b)
ax.plot((vaclog["Elapsed time"]/3600),vaclog["injection 100mbar"],marker=".", markersize=5,label='inj volume pressure')
ax.plot((vaclog["Elapsed time"]/3600),y_fit, "--",color="red", label="Y= -ax + b fit") 
ax.legend(loc="upper right")

ax.set_xlabel('Elapsed time (h)')
ax.set_ylabel('Pressure (mbar)')
ax.set_yscale('linear')
plt.title("Injection pressure evolution 100mbar conductance gauge")

Text(0.5, 1.0, 'Injection pressure evolution 100mbar conductance gauge')

## Point-by-point desorption

In [202]:
#make a sub-df from mid that includes only mass 2 data
h2 = mid.loc[:, ["mass 2.0_time", "mass 2.0_value"]]
#rename columns
h2 = h2.rename(columns={"mass 2.0_time": "datetime","mass 2.0_value": "Ion current"}).reset_index(drop=True)

print(h2)

                     datetime   Ion current
0     2023/01/13 14:48:15.864  3.501160e-10
1     2023/01/13 14:48:22.260  3.574470e-10
2     2023/01/13 14:48:28.655  3.623710e-10
3     2023/01/13 14:48:35.050  3.698370e-10
4     2023/01/13 14:48:41.445  3.766090e-10
...                       ...           ...
1692  2023/01/13 17:48:39.246  2.837350e-11
1693  2023/01/13 17:48:45.641  2.830790e-11
1694  2023/01/13 17:48:52.036  2.827990e-11
1695  2023/01/13 17:48:58.431  2.821060e-11
1696  2023/01/13 17:49:04.826  2.820090e-11

[1697 rows x 2 columns]


In [203]:
# Convert the 'time' column to datetime index for both dataframes
hv.index = (pd.to_datetime(hv['datetime']).dt.round(freq="S"))
h2.index = (pd.to_datetime(h2['datetime']).dt.round(freq="S"))
#print(hv)

# Reindex df1 to match the datetime index of df2
hv = hv.reindex(h2.index, method='nearest', limit=1)
print(hv)
#Drop the "datetime" columns
hv.drop('datetime', axis=1, inplace=True)
h2.drop('datetime', axis=1, inplace=True)

# Now df1 and df2 have the same datetime index and can be joined or aligned
# by joining on the index
resampled_df = hv.merge(h2,left_index=True, right_index=True, how="inner")

#print("aligned",resampled_df)
print(resampled_df.describe())

                               datetime        Date  Elapsed time  \
datetime                                                            
2023-01-13 14:48:16                 NaT         NaN           NaN   
2023-01-13 14:48:22                 NaT         NaN           NaN   
2023-01-13 14:48:29                 NaT         NaN           NaN   
2023-01-13 14:48:35                 NaT         NaN           NaN   
2023-01-13 14:48:41                 NaT         NaN           NaN   
...                                 ...         ...           ...   
2023-01-13 17:48:39 2023-01-13 17:48:39  13/01/2023     10663.674   
2023-01-13 17:48:46 2023-01-13 17:48:46  13/01/2023     10670.874   
2023-01-13 17:48:52 2023-01-13 17:48:52  13/01/2023     10676.878   
2023-01-13 17:48:58 2023-01-13 17:48:58  13/01/2023     10682.885   
2023-01-13 17:49:05 2023-01-13 17:49:05  13/01/2023     10690.185   

                             Time   VMon1   VMon2    VMon3    VMon4    VMon5  \
datetime              

In [204]:
#Filtering background signal for interpolated datasets

#From the signal, get the partial pressures 
h2_sens = 2.13e-2 #A/mbar
p_h2_ip = resampled_df["mass 2 corr"]/(h2_sens*0.46) #H2 equivalent pressures

#account for the thermal transpiration factor
t_delta = 4.2
T1 = 296.5
T_coef = np.sqrt(t_delta/T1)
p_h2_ip = p_h2_ip*T_coef 

#insert partial pressures into the dataframe
resampled_df.insert(22,"hydrogen pp",p_h2_ip)


In [205]:
resampled_df.fillna(0,inplace=True)
resampled_df.describe()

,Elapsed time,VMon1,VMon2,VMon3,VMon4,VMon5,VMon6,VMon7,VMon8,IMon1,...,IMon5,IMon6,IMon7,IMon8,Unnamed: 19,hv_grid,hydrogen pp,I_em,Ion current,mass 2 corr
count,1697.000000,1697.000000,1697.000000,1697.000000,1697.000000,1697.000000,1697.000000,1697.0,1697.000000,1697.000000,...,1697.000000,1697.000000,1697.000000,1697.000000,1697.0,1.697000e+03,1.697000e+03,1.697000e+03,1.697000e+03,1.697000e+03
mean,5267.186705,137.803147,97.628898,974.787342,974.721179,972.719811,484.974154,0.0,0.020837,-0.277334,...,9.861978,9.870292,-0.019815,-0.011070,0.0,2.890022e-07,2.473906e-09,2.093900e-06,2.037199e-10,2.036612e-10
std,3132.864745,223.177009,15.392424,156.480922,157.062550,160.130781,85.194371,0.0,0.005231,0.197301,...,1.797958,1.803746,0.002623,0.001663,0.0,1.797655e-07,6.402917e-09,1.372714e-06,5.271121e-10,5.271121e-10
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,-0.475200,...,-0.001400,0.000000,-0.022700,-0.013900,0.0,0.000000e+00,3.418474e-10,0.000000e+00,2.820090e-11,2.814216e-11
25%,2552.128000,0.100000,100.060000,1000.020000,1000.140000,1000.120000,499.960000,0.0,0.020000,-0.424000,...,9.911600,9.920800,-0.020800,-0.011800,0.0,2.060000e-08,5.021827e-10,1.840000e-08,4.140030e-11,4.134156e-11
50%,5267.677000,0.120000,100.060000,1000.020000,1000.160000,1000.140000,499.960000,0.0,0.020000,-0.403400,...,10.352200,10.363500,-0.019900,-0.011300,0.0,4.034000e-07,6.250925e-10,3.036700e-06,5.151870e-11,5.145996e-11
75%,7978.786000,499.940000,100.080000,1000.040000,1000.160000,1000.140000,499.980000,0.0,0.020000,0.016900,...,10.620100,10.630800,-0.019200,-0.010500,0.0,4.240000e-07,3.065728e-09,3.073100e-06,2.524410e-10,2.523823e-10
max,10690.185000,499.980000,100.200000,1000.040000,1000.180000,1000.160000,500.000000,0.0,0.040000,0.464700,...,11.038800,11.053600,0.000000,0.000000,0.0,4.752000e-07,6.550687e-08,3.577600e-06,5.392830e-09,5.392771e-09


# Apparent desorption calculation method
### Elena's formula for the cryo setup
$$
\eta=\frac{C_{H2}\cdot (\Delta p_1 - \Delta p_2)q_e}{k_B\cdot T \cdot I_e}
$$

Where $\Delta p_i=p(t)_i - p(t=0)_i$ and $i=1,2$
In the case of the 4K setup, we know the exact conductance of gasses. The formula above applies well.

Due to the EGA being fully retracted, it is safe to assume that the desorption takes place at approximately room temperature

In [206]:
#calculation delta_p-s
dp1 = (np.mean(p_h2)-np.mean(base_h2))*100
print(dp1)

#Constants
k_B = 1.38e-23
T = 293
print(T)
q_e = 1.6e-19
c_h2 = 37.91/1000
s_h2=42/1000
x_h2=1/((1/c_h2)+(1/s_h2))
print(x_h2)
#Emission current

I_em = np.nanmean(resampled_df["I_em"].nlargest(30))
print(I_em)

-7.189471644682858e-08
293
0.01992516581153798
3.5198799999999994e-06


In [207]:
#Desorption yield
eta_h2 = (x_h2*dp1*q_e)/(k_B*T*I_em)
#eta_co = (c_co*(dp1-dp2)*q_e)/(k_B*T*I_em)
print("The desorption yield for H2 [mol/el]:",eta_h2,"\n",)#""The desorption yield for CO2 [mol/el]:",eta_co2)

The desorption yield for H2 [mol/el]: -0.016104392158163536 



## Desorption yield point-by-point

$$
\eta=\frac{C_{H2}\cdot (p_{n} - p_{n-1}) \cdot q_e}{k_B\cdot T \cdot I_{n}}
$$


In [208]:
#partial pressure delta

p_delta = resampled_df["hydrogen pp"].diff().fillna(resampled_df["hydrogen pp"])*100
p_delta = p_delta.clip(lower=0)
print(p_delta)
#hv_shifted = resampled_df.shift(periods=1)
#print(hv_shifted)
#p_delta = resampled_df["hydrogen pp"] - hv_shifted["hydrogen pp"]
#print(resampled_df["hydrogen pp"].describe())
#print(hv_shifted["hydrogen pp"].describe())
print(p_delta.describe())

datetime
2023-01-13 14:48:16    4.252203e-07
2023-01-13 14:48:22    8.905085e-09
2023-01-13 14:48:29    5.981263e-09
2023-01-13 14:48:35    9.069072e-09
2023-01-13 14:48:41    8.226059e-09
                           ...     
2023-01-13 17:48:39    0.000000e+00
2023-01-13 17:48:46    0.000000e+00
2023-01-13 17:48:52    0.000000e+00
2023-01-13 17:48:58    0.000000e+00
2023-01-13 17:49:05    0.000000e+00
Name: hydrogen pp, Length: 1697, dtype: float64
count    1.697000e+03
mean     4.214024e-09
std      3.497321e-08
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      9.669142e-11
max      6.158247e-07
Name: hydrogen pp, dtype: float64


In [209]:
#constants are already defined
#emission current and H2 partial pressures are now available at the same sampling rates
print(resampled_df.describe())
I_n = resampled_df["I_em"].replace(0, np.nan).dropna()

print(I_n)
#I_n.where(em <= 1.5e-6, np.nan, inplace=True)

eta = ((x_h2*p_delta*q_e)/(k_B*T*I_n)).dropna()
eta = eta.loc[eta.index[1]:]
print("desorption yields point-by-point:","\n", eta)

       Elapsed time        VMon1        VMon2        VMon3        VMon4  \
count   1697.000000  1697.000000  1697.000000  1697.000000  1697.000000   
mean    5267.186705   137.803147    97.628898   974.787342   974.721179   
std     3132.864745   223.177009    15.392424   156.480922   157.062550   
min        0.000000     0.000000     0.000000     0.000000     0.000000   
25%     2552.128000     0.100000   100.060000  1000.020000  1000.140000   
50%     5267.677000     0.120000   100.060000  1000.020000  1000.160000   
75%     7978.786000   499.940000   100.080000  1000.040000  1000.160000   
max    10690.185000   499.980000   100.200000  1000.040000  1000.180000   

             VMon5        VMon6   VMon7        VMon8        IMon1  ...  \
count  1697.000000  1697.000000  1697.0  1697.000000  1697.000000  ...   
mean    972.719811   484.974154     0.0     0.020837    -0.277334  ...   
std     160.130781    85.194371     0.0     0.005231     0.197301  ...   
min       0.000000     0.000

In [210]:
#slicing eta to match resampled_df["Elapsed time"]:
# Slice x using the index of y
hv_downsampled = resampled_df.loc[eta.index, :]

# Plotting

In [211]:
#define parameters for plotting
#annotation params
font = dict(size = "x-small", color ="green", style ="italic",rotation="vertical")

plt.rcParams["figure.figsize"] = [8,5]
plt.rcParams["axes.edgecolor"] = "black"
plt.rcParams["axes.grid"] = True
plt.rcParams["grid.color"] = "black"
plt.rcParams["grid.linewidth"] = 0.35

In [212]:
#plot the aligned dataframe results
plt.figure()
plt.plot(resampled_df["Elapsed time"],resampled_df["Ion current"],linestyle="-",marker=".", markersize=4,label='H2 ion current')
plt.plot(resampled_df["Elapsed time"],resampled_df["I_em"],linestyle="-",marker=".", markersize=4,label='Emission current')
plt.xlabel('Elapsed time (s)')
plt.ylabel('Current (A)')
plt.legend(loc="upper right")
plt.title("S4 ion current and emission current")


plt.yscale('log')


In [213]:
#Plotting the all the relevant MID spectrums
plt.figure()

plt.plot(new_df["Timestamp, mass 2"],mid["mass 2.0_value"],marker=".", markersize=4,label='M2')
plt.plot(new_df["Timestamp, mass 4"],mid["mass 4.0_value"],marker=".", markersize=4,label='M4')
plt.plot(new_df["Timestamp, mass 12"],mid["mass 12.0_value"],marker=".", markersize=4,label='M12')
plt.plot(new_df["Timestamp, mass 15"],mid["mass 15.0_value"],marker=".", markersize=4,label='M16')
plt.plot(new_df["Timestamp, mass 16"],mid["mass 16.0_value"],marker=".", markersize=4,label='M15')
plt.plot(new_df["Timestamp, mass 18"],mid["mass 18.0_value"],marker=".", markersize=4,label='M18')
plt.plot(new_df["Timestamp, mass 28"],mid["mass 28.0_value"],marker=".", markersize=4,label='M28')
plt.plot(new_df["Timestamp, mass 32"],mid["mass 32.0_value"],marker=".", markersize=4,label='M32')
plt.plot(new_df["Timestamp, mass 40"],mid["mass 40.0_value"],marker=".", markersize=4,label='M40')
# plt.plot(new_df["Timestamp, mass 44"],mid["mass 44.0_value"],marker=".", markersize=4,label='M44')

plt.xlabel('Timestamp')
plt.ylabel('Ion current (A)')
plt.legend(title="Mol masses", loc="upper right")
plt.title("S1 MID scan")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(14))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')

#write comments
trans  = transforms.blended_transform_factory(
    ax.transData, ax.transAxes)
for i in hv_comm.index:
    plt.text(hv_comm["datetime"][i],0.95,str(hv_comm["Comment"][i]),fontdict=font, transform=trans)
    
plt.tight_layout()  # Ensure all elements are properly placed within the figure
plt.savefig(r'./graphs/MID.png',dpi=300)
plt.show()


In [214]:
#plotting HiVolta log
plt.figure()

plt.plot(resampled_df["Elapsed time"],resampled_df["hv_grid"],marker=".", markersize=4,label='Grid current')
plt.plot(resampled_df["Elapsed time"],resampled_df["I_em"],marker=".", markersize=4,label='Emission current')
plt.xlabel('Timestamp')
plt.ylabel('Current (A)')
plt.legend(loc="lower left")
plt.title("S4 HiVolta log")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

    
#comments
trans  = transforms.blended_transform_factory(
    ax.transData, ax.transAxes)
for i in hv_comm.index:
    plt.text(hv_comm["datetime"][i],0.8,str(hv_comm["Comment"][i]),fontdict=font, transform=trans)
      
plt.tight_layout()  # Ensure all elements are properly placed within the figure
plt.savefig(r'./graphs/HiVolta.png',dpi=300)
plt.show()


In [215]:
#Plotting BA2, BA1 pressure
plt.figure()
plt.plot(vaclog["Time"],vaclog["Barion_2"],marker=".", markersize=4,label='Barion 2')
plt.plot(vaclog["Time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')
plt.xlabel('Timestamp')
plt.ylabel('Pressure (mbar)')
plt.legend()
plt.title("S4 Total pressure evolution")
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')
plt.yscale('log')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')    
     
plt.tight_layout()  # Ensure all elements are properly placed within the figure  
plt.savefig(r'./graphs/Barions.png',dpi=300)
plt.show()

In [216]:
## Plotting the temp evolution
plt.figure()
plt.plot((vaclog["Time"]),polyfit(A,vaclog["T-CERNOX"]),marker=".", markersize=5,label='T-CERNOX')          
plt.xlabel('Timestamp')
plt.ylabel('Temperature (K)')
plt.legend()
plt.title("S4 Temperature evolution")
plt.yscale('linear')
plt.tick_params(axis="y", which='minor')
plt.grid(which='minor', axis='y')

ax = plt.gca()
ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
for label in ax.get_xticklabels(which='major'):
    label.set(rotation=30, horizontalalignment='right')
    
plt.tight_layout()  # Ensure all elements are properly placed within the figure    
plt.savefig(r'./graphs/Temp.png',dpi=300)
plt.show()

In [217]:
#plotting point-by-point eta
plt.figure(figsize=(10,6))
plt.plot(hv_downsampled["Elapsed time"],eta,marker=".", markersize=4,label='eta')
#plt.plot(vaclog["Elapsed time"],vaclog["Barion_1"],marker=".", markersize=4,label='Barion 1')

plt.xlabel('Elapsed time (s)')
plt.ylabel('Electon stimulated desorption yield (M/el)')
plt.legend()
plt.title("S4 ESD yield, 2e19 M surface pre-injection,")
ax = plt.gca()
ax.set_yscale('log')
ax.grid(which="major",axis="x")
plt.tick_params(axis="y", which='minor')
plt.grid(visible=None, which='both', axis='both')

plt.tight_layout()  # Ensure all elements are properly placed within the figure    
plt.savefig(r'./graphs/Point-by-point ESD yield.png',dpi=300)
plt.show()